<center><br><br>
<font size=6>🎓 <b>Advanced Deep Learning - NLP Final Project</b></font><br>
<font size=6>⚖️  <b>Training - microsoft/mdeberta-v3-base EX5</b></font><br>
<font size=5>👥 <b>Group W</b></font><br><br>
<b>Adi Shalit</b>, ID: <code>206628885</code><br>
<b>Gal Gussarsky</b>, ID: <code>206453540</code><br><br>
<font size=4>📘 Course ID: <code>05714184</code></font><br>
<font size=4>📅 Spring 2025</font>
<br><br>
<hr style="width:60%; border:1px solid gray;"></center>


# 📑 Table of Contents

- [Training](#Training)
- [ Load best Model & Test](#Load-Best-Model)




In [ ]:
# # === Compatible versions ===
# !pip install --upgrade --force-reinstall --no-cache-dir \
#   transformers==4.43.3 \
#   accelerate==0.30.1 \
#   datasets==2.20.0 \
#   evaluate==0.4.2 \
#   optuna==3.6.1 \
#   wandb==0.17.5 \
#   fsspec==2024.5.0 \
#   gcsfs==2024.5.0


In [ ]:
# !pip install transformers==4.44.2 accelerate==0.34.2

# from transformers import Trainer
# print("✅ Trainer import successful")


## Load Dataset & Imports

In [7]:


# # -------------------------
# # Colab Drive setup
# # -------------------------
# drive.mount("/content/drive")
# DRIVE_OUT_DIR = "/content/drive/MyDrive/adv_dl_models2"
DRIVE_OUT_DIR = "adv_dl_models2"
os.makedirs(DRIVE_OUT_DIR, exist_ok=True)

train_path = "Corona_NLP_train_cleaned_translated.csv"
test_path  = "Corona_NLP_test_cleaned_translated.csv"

df_train = pd.read_csv(train_path)
df_test  = pd.read_csv(test_path)

print(df_train.shape, df_test.shape)
df_train.head()


(41157, 7) (3798, 7)


UserName  ScreenName   Location     TweetAt  \
0      3799       48751     London  16-03-2020   
1      3800       48752         UK  16-03-2020   
2      3801       48753  Vagabonds  16-03-2020   
3      3802       48754        NaN  16-03-2020   
4      3803       48755        NaN  16-03-2020   

                                       OriginalTweet           Sentiment  \
0            @MeNyrbie @Phil_Gahan @Chrisitv and and             Neutral   
1  advice Talk to your neighbours family to excha...            Positive   
2  covid Australia: Woolworths to give elderly, d...            Positive   
3  My food stock is not the only one which is emp...            Positive   
4  Me, ready to go at supermarket during the covi...  Extremely Negative   

  DetectedLang  
0           en  
1           en  
2           en  
3           en  
4           en

# 📊 RoBERTa – Hyperparameter Exploration (Trainer + Optuna + W&B)

In this stage, we continue our experimentation with **RoBERTa-base** for 5-class sentiment classification.  
Based on our previous findings, **a higher number of unfrozen layers** tends to improve performance.  

We now design a structured **Optuna hyperparameter search** to explore:
- Learning rate (`1e-6 → 3e-5`)  
- Weight decay (`1e-6 → 1e-2`)  
- Batch sizes (`[4, 8, 16, 32]`)  
- Number of unfrozen layers (`5 → 12`)  

Training is managed through the **🤗 HuggingFace Trainer API**, with:  
- **W&B logging** for live monitoring  
- **Early stopping** (`patience=4`)  
- **Fixed epochs = 12**  
- Evaluation metric = **Macro F1**  

This approach allows us to balance **model expressiveness** (more unfrozen layers) with **computational efficiency**, while ensuring reproducibility through deterministic seeding.  


# Training

In [3]:
# =========================
# ADV DL – Part B: RoBERTa (Trainer API + Optuna + W&B)
# Freeze base, unfreeze last k; 5-class sentiment
# Expects df_train/df_test already loaded with columns: ["OriginalTweet","Sentiment"]
# =========================

import os, math, random, time, json
from typing import Dict, Any
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report, confusion_matrix

import torch
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    DataCollatorWithPadding, TrainingArguments, Trainer,
    EarlyStoppingCallback
)
import optuna
import wandb
from datasets import Dataset, DatasetDict
import numpy as np
import datasets

# Patch datasets' arrow → numpy conversion to be NumPy 2.x safe
old_arrow_array_to_numpy = datasets.formatting.formatting.NumpyArrowExtractor._arrow_array_to_numpy

def safe_arrow_array_to_numpy(self, pa_array):
    return np.asarray(pa_array)  # allows copy when needed

datasets.formatting.formatting.NumpyArrowExtractor._arrow_array_to_numpy = safe_arrow_array_to_numpy


# -------------------------
# Globals
# -------------------------
MODEL_NAME = "roberta-base"
MAX_LEN = 512
PROJECT = "adv-dl2-p1"
BASE_RUN_NAME = "roberta-base_trainer"
TRIALS = 12
SEED = 42
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

def set_seed(seed=42):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(SEED)

# -------------------------
# Labels (5 classes)
# -------------------------
CANON = {
    "extremely negative": "extremely negative",
    "negative": "negative",
    "neutral": "neutral",
    "positive": "positive",
    "extremely positive": "extremely positive",
}
ORDER = ["extremely negative","negative","neutral","positive","extremely positive"]
LABEL2ID = {lab: i for i, lab in enumerate(ORDER)}
ID2LABEL = {i: lab for lab, i in LABEL2ID.items()}

def normalize_label(s: str) -> str:
    s = str(s).strip().lower()
    s = s.replace("very negative", "extremely negative")
    s = s.replace("very positive", "extremely positive")
    s = s.replace("extreme negative", "extremely negative")
    s = s.replace("extreme positive", "extremely positive")
    return CANON.get(s, s)

# -------------------------
# Expect df_train/df_test preloaded
# -------------------------
assert "OriginalTweet" in df_train.columns and "Sentiment" in df_train.columns, "df_train missing required columns"
assert "OriginalTweet" in df_test.columns and "Sentiment" in df_test.columns, "df_test missing required columns"

def prep_df(df: pd.DataFrame) -> pd.DataFrame:
    df = df.dropna(subset=["OriginalTweet", "Sentiment"]).copy()
    df["text"] = df["OriginalTweet"].astype(str).str.strip()
    df["label_name"] = df["Sentiment"].apply(normalize_label)
    df = df[df["label_name"].isin(ORDER)].reset_index(drop=True)
    df["label"] = df["label_name"].map(LABEL2ID)
    return df[["text","label","label_name"]]

dftrain_ = prep_df(df_train)
dftest_  = prep_df(df_test)

# Stratified split
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(
    dftrain_, test_size=0.1, stratify=dftrain_["label"], random_state=SEED
)
print(f"Train/Val/Test sizes: {len(train_df)}/{len(val_df)}/{len(dftest_)}")

# -------------------------
# HF Datasets + Tokenizer
# -------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

def tokenize_batch(batch):
    return tokenizer(batch["text"], truncation=True, max_length=MAX_LEN)

ds_train = Dataset.from_pandas(train_df.reset_index(drop=True))
ds_val   = Dataset.from_pandas(val_df.reset_index(drop=True))
ds_test  = Dataset.from_pandas(dftest_.reset_index(drop=True))

ds = DatasetDict({"train": ds_train, "validation": ds_val, "test": ds_test})
ds = ds.remove_columns([c for c in ds["train"].column_names if c not in ["text","label"]])
ds_tok = ds.map(tokenize_batch, batched=True, remove_columns=["text"])
ds_tok = ds_tok.rename_column("label", "labels")   # Trainer expects 'labels'
ds_tok.set_format("torch")

collator = DataCollatorWithPadding(tokenizer=tokenizer)

# -------------------------
# Model + freezing policy
# -------------------------
def build_model(unfreeze_last_k: int = 4) -> AutoModelForSequenceClassification:
    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_NAME, num_labels=len(ORDER), id2label=ID2LABEL, label2id=LABEL2ID
    )
    base = getattr(model, "roberta", None) or getattr(model, "bert", None)
    if base is not None:
        for p in base.parameters(): p.requires_grad = False  # freeze all
        if hasattr(base, "encoder") and hasattr(base.encoder, "layer") and unfreeze_last_k > 0:
            for layer in base.encoder.layer[-unfreeze_last_k:]:
                for p in layer.parameters(): p.requires_grad = True
    if hasattr(model, "classifier"):
        for p in model.classifier.parameters(): p.requires_grad = True
    return model

# -------------------------
# Metrics (pure sklearn to avoid evaluate/datasets mismatch)
# -------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    p, r, f1, _ = precision_recall_fscore_support(labels, preds, average="macro", zero_division=0)
    return {
        "accuracy": acc,
        "precision_macro": p,
        "recall_macro": r,
        "f1_macro": f1,
    }

# -------------------------
# W&B helper
# -------------------------
def log_conf_mat_to_wandb(y_true, y_pred, prefix="val"):
    table = wandb.plot.confusion_matrix(
        y_true=y_true,
        preds=y_pred,
        class_names=[ID2LABEL[i] for i in range(len(ID2LABEL))]
    )
    wandb.log({f"{prefix}/confusion_matrix": table})

# -------------------------
# Optuna objective (wraps Trainer)
# -------------------------
FIXED_EPOCHS = 12
PATIENCE = 4
LOG_STEPS = 20

def make_trainer(params: Dict[str, Any], output_dir: str, run_name: str) -> Trainer:
    model = build_model(unfreeze_last_k=int(params["unfreeze_last_k"]))
    args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy="epoch",         # canonical name
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="eval_f1_macro",  # Trainer prefixes eval metrics
        greater_is_better=True,

        num_train_epochs=FIXED_EPOCHS,
        learning_rate=float(params["lr"]),
        weight_decay=float(params["weight_decay"]),
        warmup_ratio=float(params["warmup_ratio"]),
        per_device_train_batch_size=int(params["per_device_train_batch_size"]),
        per_device_eval_batch_size=int(params["per_device_eval_batch_size"]),
        gradient_accumulation_steps=int(params["gradient_accumulation_steps"]),
        fp16=torch.cuda.is_available(),

        logging_steps=LOG_STEPS,
        report_to=["wandb"],
        run_name=run_name,
        seed=SEED,
        save_total_limit=2,
    )
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=ds_tok["train"],
        eval_dataset=ds_tok["validation"],
        tokenizer=tokenizer,
        data_collator=collator,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=PATIENCE)]
    )
    return trainer
def objective(trial: optuna.trial.Trial) -> float:
    params = {
        "lr": trial.suggest_float("lr", 1e-6, 3e-5, log=True),  # safer upper bound for more layers
        "weight_decay": trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True),  # updated upper bound
        "warmup_ratio": 0.06,  # fixed
        "batch_size": trial.suggest_categorical("batch_size", [4, 8, 16, 32]),  # single batch size for both train & eval
        "gradient_accumulation_steps": 1,  # fixed
        "unfreeze_last_k": trial.suggest_int("unfreeze_last_k", 5, 12),  # between 5 and 12 layers unfrozen
    }

    run_name = f"{BASE_RUN_NAME}_trial_{trial.number}"
    output_dir = os.path.join(DRIVE_OUT_DIR, run_name)

    wandb.init(project=PROJECT, name=run_name, config=params, reinit=True)

    # pass the same batch_size for both train & eval
    trainer = make_trainer(
        {
            **params,
            "per_device_train_batch_size": params["batch_size"],
            "per_device_eval_batch_size": params["batch_size"],
        },
        output_dir,
        run_name
    )

    _ = trainer.train()  # early stopping inside

    # Evaluate (best model auto-loaded)
    metrics = trainer.evaluate()
    wandb.log({
        "val/accuracy":        metrics.get("eval_accuracy"),
        "val/precision_macro": metrics.get("eval_precision_macro"),
        "val/recall_macro":    metrics.get("eval_recall_macro"),
        "val/f1_macro":        metrics.get("eval_f1_macro"),
    })

    # Confusion matrix on validation
    preds = trainer.predict(ds_tok["validation"])
    y_true = preds.label_ids
    y_pred = np.argmax(preds.predictions, axis=-1)
    log_conf_mat_to_wandb(y_true, y_pred, prefix="val")

    # Save best checkpoint path to W&B summary
    best_dir = trainer.state.best_model_checkpoint
    wandb.summary["best_checkpoint_path"] = best_dir if best_dir else output_dir

    wandb.finish()

    # Return the metric Optuna maximizes
    return float(metrics.get("eval_f1_macro", 0.0))
# -------------------------
# Optuna search (RESUMABLE)
# -------------------------
storage_url = f"sqlite:///{os.path.join(DRIVE_OUT_DIR, f'{BASE_RUN_NAME}_optuna.db')}"
study_name = f"{BASE_RUN_NAME}_study"

try:
    # load_study only needs study_name and storage
    study = optuna.load_study(study_name=study_name, storage=storage_url)
    print(f"Loaded existing study: {study_name}")
except KeyError:
    study = optuna.create_study(
        study_name=study_name,
        storage=storage_url,
        direction="maximize"
    )
    print(f"Created new study: {study_name}")

completed = len([t for t in study.get_trials(deepcopy=False) if t.state == optuna.trial.TrialState.COMPLETE])
remaining = max(0, TRIALS - completed)
print(f"Trials completed: {completed} / {TRIALS} → remaining: {remaining}")

if remaining > 0:
    study.optimize(objective, n_trials=remaining, show_progress_bar=True)
else:
    print("Requested number of trials already completed; skipping optimization.")

print("Best trial:", study.best_trial.number, "F1_macro:", study.best_value)

best_params = study.best_params
best_params["per_device_train_batch_size"] = best_params["batch_size"]
best_params["per_device_eval_batch_size"] = best_params["batch_size"]



# -------------------------
# Retrain best config cleanly
# -------------------------
best_run_name = f"{BASE_RUN_NAME}_best"
best_out_dir = os.path.join(DRIVE_OUT_DIR, best_run_name)
wandb.init(project=PROJECT, name=best_run_name, config=best_params, reinit=True)

trainer = make_trainer(best_params, best_out_dir, best_run_name)
trainer.train()

best_ckpt_dir = trainer.state.best_model_checkpoint or best_out_dir
print("Best checkpoint directory:", best_ckpt_dir)
wandb.summary["best_checkpoint_path"] = best_ckpt_dir

# -------------------------
# # Final TEST evaluation
# # -------------------------
# test_metrics = trainer.evaluate(ds_tok["test"])

# preds_test = trainer.predict(ds_tok["test"])
# y_true_test = preds_test.label_ids
# y_pred_test = np.argmax(preds_test.predictions, axis=-1)

# acc = accuracy_score(y_true_test, y_pred_test)
# p, r, f1, _ = precision_recall_fscore_support(y_true_test, y_pred_test, average="macro", zero_division=0)
# print(f"\nTEST | acc={acc:.4f} | f1_macro={f1:.4f} | precision_macro={p:.4f} | recall_macro={r:.4f}\n")

# print("Per-class report:")
# print(classification_report(
#     y_true_test, y_pred_test,
#     target_names=[ID2LABEL[i] for i in range(len(ORDER))],
#     zero_division=0
# ))

# # Log to W&B
# report = classification_report(
#     y_true_test, y_pred_test,
#     target_names=[ID2LABEL[i] for i in range(len(ORDER))],
#     zero_division=0, output_dict=True
# )

# payload = {
#     "test/accuracy": acc,
#     "test/precision_macro": p,
#     "test/recall_macro": r,
#     "test/f1_macro": f1,
# }
# for cls_name in ORDER:
#     if cls_name in report:
#         payload[f"test/{cls_name}/precision"] = report[cls_name]["precision"]
#         payload[f"test/{cls_name}/recall"]    = report[cls_name]["recall"]
#         payload[f"test/{cls_name}/f1"]        = report[cls_name]["f1-score"]

# wandb.log(payload)
# table = wandb.plot.confusion_matrix(
#     y_true=y_true_test,
#     preds=y_pred_test,
#     class_names=[ID2LABEL[i] for i in range(len(ID2LABEL))]
# )
# wandb.log({"test/confusion_matrix": table})
# wandb.finish()

# # Save the final best model weights to Drive
# final_path = os.path.join(DRIVE_OUT_DIR, f"best_{BASE_RUN_NAME}_trainer.pt")
# torch.save(trainer.model.state_dict(), final_path)
# print("Saved best model state_dict to:", final_path)


Train/Val/Test sizes: 37039/4116/3798


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

[I 2025-08-15 07:11:01,240] Trial 11 finished with value: 0.8600960158069137 and parameters: {'lr': 2.960627147251281e-05, 'weight_decay': 0.008806331558971375, 'batch_size': 16, 'unfreeze_last_k': 12}. Best is trial 11 with value: 0.8600960158069137.


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


[I 2025-08-15 07:54:33,513] Trial 12 finished with value: 0.852805045977085 and parameters: {'lr': 2.892824570696833e-05, 'weight_decay': 0.00979168138442126, 'batch_size': 16, 'unfreeze_last_k': 12}. Best is trial 11 with value: 0.8600960158069137.
Best trial: 11 F1_macro: 0.8600960158069137


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


KeyError: 'warmup_ratio'

# ✅ Final Training – Best Optuna Trials

We selected the **top-performing configurations** from our Optuna study for RoBERTa-base with high unfrozen layers.  

### 🔬 Best Trials and Hyperparameters
- **Trial 12** → `{'lr': 2.8928e-05, 'weight_decay': 9.79e-03, 'batch_size': 16, 'unfreeze_last_k': 12}`  
  → **F1_macro = 0.8528**  

- **Trial 11** → `{'lr': 2.9606e-05, 'weight_decay': 8.81e-03, 'batch_size': 16, 'unfreeze_last_k': 12}`  
  → **F1_macro = 0.8601**  🏆 **Best**  

- **Trial 4** → `{'lr': 2.1162e-05, 'weight_decay': 1.33e-03, 'batch_size': 16, 'unfreeze_last_k': 10}`  
  → **F1_macro = 0.8327**  

### 📈 Summary
- The **best-performing trial was #11**, achieving **F1_macro = 0.8601**,  
  confirming that **larger numbers of unfrozen layers (k=12)** and carefully tuned learning rates yield the strongest results.  
- Both Trial 11 and Trial 12 converge to high performance with similar hyperparameters,  
  while Trial 4 (with only 10 unfrozen layers) lagged behind.  

➡️ We now compare all 3 models with **final evaluation**.  


# Load Best Model

In [7]:
import os
import torch
import pandas as pd
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report
from google.colab import drive

# ====== DRIVE SETUP ======
# drive.mount("/content/drive")
# SAVE_DIR = "/content/drive/MyDrive/adv_dl_models_final2_best"
SAVE_DIR = "adv_dl_models_final2_best"
os.makedirs(SAVE_DIR, exist_ok=True)

# ====== CLEANING FUNCTION ======
def prep_df(df):
    # Drop NaNs in text and label column
    df = df.dropna(subset=["OriginalTweet", "Sentiment"]).copy()
    # Make sure text is string
    df["OriginalTweet"] = df["OriginalTweet"].astype(str).str.strip()
    return df

df_train = prep_df(df_train)
df_test = prep_df(df_test)

# ====== MAP SENTIMENT TO LABELS ======
label2id = {label: idx for idx, label in enumerate(df_train["Sentiment"].unique())}
id2label = {idx: label for label, idx in label2id.items()}

df_train["label"] = df_train["Sentiment"].map(label2id)
df_test["label"]  = df_test["Sentiment"].map(label2id)

# ====== SPLIT ======
df_train_split, df_val_split = train_test_split(
    df_train,
    test_size=0.1,
    random_state=42,
    stratify=df_train["label"]
)

# ====== TOKENIZER ======
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def tokenize(batch):
    return tokenizer(batch["OriginalTweet"], padding="max_length", truncation=True)

train_dataset = Dataset.from_pandas(df_train_split).map(tokenize, batched=True)
val_dataset   = Dataset.from_pandas(df_val_split).map(tokenize, batched=True)
test_dataset  = Dataset.from_pandas(df_test).map(tokenize, batched=True)

for ds in [train_dataset, val_dataset, test_dataset]:
    ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

# ====== METRICS ======
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

# ====== BEST PARAMS ======
best_params_list = [
    {'lr': 2.892824570696833e-05, 'weight_decay': 0.00979168138442126, 'batch_size': 16, 'unfreeze_last_k': 12},
    {'lr': 2.960627147251281e-05, 'weight_decay': 0.008806331558971375, 'batch_size': 16, 'unfreeze_last_k': 12},
    {'lr': 2.116177565568772e-05, 'weight_decay': 0.0013266583844993018, 'batch_size': 16, 'unfreeze_last_k': 10}
]

# ====== TRAIN/EVAL LOOP ======
for i, params in enumerate(best_params_list, start=1):
    print(f"\n=== Training model {i} with params: {params} ===")

    # Load base model
    model = RobertaForSequenceClassification.from_pretrained(
        "roberta-base",
        num_labels=len(label2id),
        id2label=id2label,
        label2id=label2id
    )

    # Freeze all except last k layers
    for param in model.roberta.parameters():
        param.requires_grad = False
    for layer in model.roberta.encoder.layer[-params["unfreeze_last_k"]:]:
        for param in layer.parameters():
            param.requires_grad = True

    # Training args
    training_args = TrainingArguments(
        output_dir=f"./results_model_{i}",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=1,
        learning_rate=params["lr"],
        per_device_train_batch_size=params["batch_size"],
        per_device_eval_batch_size=params["batch_size"],
        weight_decay=params["weight_decay"],
        num_train_epochs=12,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        greater_is_better=True,
        logging_dir=f"./logs_model_{i}",
        logging_strategy="epoch",
        seed=42
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
    )

    # Train
    trainer.train()

    # Evaluate on test
    test_metrics = trainer.evaluate(test_dataset)
    print(f"Test metrics for model {i}: {test_metrics}")

    # Per-class classification report
    preds_output = trainer.predict(test_dataset)
    y_true = preds_output.label_ids
    y_pred = preds_output.predictions.argmax(axis=-1)
    print(f"\nClassification report for model {i}:\n")
    print(classification_report(
        y_true, y_pred,
        target_names=[id2label[idx] for idx in sorted(id2label.keys())],
        digits=4
    ))

    # Save model weights
    save_path = os.path.join(SAVE_DIR, f"roberta_base_best_set{i}.pt")
    torch.save(model.state_dict(), save_path)
    print(f"Model {i} saved to {save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/37039 [00:00<?, ? examples/s]

Map:   0%|          | 0/4116 [00:00<?, ? examples/s]

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

...

Test metrics for model 1: {'eval_loss': 0.9277804493904114, 'eval_accuracy': 0.8267509215376514, 'eval_precision': 0.8278203596672534, 'eval_recall': 0.8267509215376514, 'eval_f1': 0.8260394900565873, 'eval_runtime': 30.0954, 'eval_samples_per_second': 126.199, 'eval_steps_per_second': 7.908, 'epoch': 12.0}

Classification report for model 1:

                    precision    recall  f1-score   support

           Neutral     0.8077    0.8481    0.8274       619
          Positive     0.8181    0.7930    0.8054       947
Extremely Negative     0.8050    0.9274    0.8619       592
          Negative     0.8456    0.7733    0.8078      1041
Extremely Positive     0.8557    0.8514    0.8536       599

          accuracy                         0.8268      3798
         macro avg     0.8264    0.8387    0.8312      3798
      weighted avg     0.8278    0.8268    0.8260      3798

Model 1 saved to /content/drive/MyDrive/adv_dl_models_final2_best/roberta_base_best_set1.pt

=== Training model

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Test metrics for model 2: {'eval_loss': 0.6522133946418762, 'eval_accuracy': 0.8354397051079515, 'eval_precision': 0.839288564650419, 'eval_recall': 0.8354397051079515, 'eval_f1': 0.8358491465662637, 'eval_runtime': 30.0756, 'eval_samples_per_second': 126.282, 'eval_steps_per_second': 7.913, 'epoch': 9.0}

Classification report for model 2:

                    precision    recall  f1-score   support

           Neutral     0.8910    0.8320    0.8605       619
          Positive     0.7811    0.8627    0.8199       947
Extremely Negative     0.8278    0.8767    0.8515       592
          Negative     0.8223    0.8136    0.8180      1041
Extremely Positive     0.9188    0.7930    0.8513       599

          accuracy                         0.8354      3798
         macro avg     0.8482    0.8356    0.8402      3798
      weighted avg     0.8393    0.8354    0.8358      3798

Model 2 saved to /content/drive/MyDrive/adv_dl_models_final2_best/roberta_base_best_set2.pt

=== Training model 3

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Test metrics for model 3: {'eval_loss': 0.7150322198867798, 'eval_accuracy': 0.8006845708267509, 'eval_precision': 0.8050816509396402, 'eval_recall': 0.8006845708267509, 'eval_f1': 0.8010498145434137, 'eval_runtime': 30.0834, 'eval_samples_per_second': 126.249, 'eval_steps_per_second': 7.911, 'epoch': 10.0}

Classification report for model 3:

                    precision    recall  f1-score   support

           Neutral     0.8664    0.7754    0.8184       619
          Positive     0.7483    0.8099    0.7779       947
Extremely Negative     0.8052    0.8868    0.8441       592
          Negative     0.7687    0.7791    0.7739      1041
Extremely Positive     0.8945    0.7646    0.8245       599

          accuracy                         0.8007      3798
         macro avg     0.8166    0.8032    0.8077      3798
      weighted avg     0.8051    0.8007    0.8010      3798

Model 3 saved to /content/drive/MyDrive/adv_dl_models_final2_best/roberta_base_best_set3.pt


# 📊 Final RoBERTa Model Comparison

| Model | Learning Rate | Weight Decay | Batch Size | Unfrozen Layers | Epochs | Accuracy | Precision | Recall | F1 Score |
|-------|---------------|--------------|------------|-----------------|--------|----------|-----------|--------|----------|
| **1** | 2.89e-05      | 9.79e-03     | 16         | 12              | 12     | 0.8268   | 0.8278    | 0.8268 | 0.8260   |
| **2 🏆** | 2.96e-05      | 8.81e-03     | 16         | 12              | 9      | **0.8354** | **0.8393** | **0.8354** | **0.8358** |
| **3** | 2.12e-05      | 1.33e-03     | 16         | 10              | 10     | 0.8007   | 0.8051    | 0.8007 | 0.8010   |

---

### 🔎 Key Insights
- **Model 2** is the best: highest **Accuracy (83.5%)** and **F1 (0.836)**.  
- **Model 1** is strong but slightly behind (82.6%).  
- **Model 3** lags (80.1%), confirming that **fewer unfrozen layers (10) reduce performance**.
